### Importing all dependencies

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re,string
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
from sklearn.model_selection import train_test_split
from nltk.corpus import wordnet
import re
import warnings
warnings.filterwarnings('ignore')

### Loading of Data

In [2]:
%%time 
df = pd.read_csv(r'IMDB-Dataset.csv',encoding='latin-1')

CPU times: total: 2.58 s
Wall time: 3.37 s


### Data Cleaning and Preprocessing

In [3]:
#Customize stopword as per data
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
new_stopwords = ["would","shall","could","might"]
stop_words.extend(new_stopwords)
stop_words.remove("not")
stop_words=set(stop_words)
print(stop_words)

{'you', 'herself', "won't", 'down', 'or', "that'll", 'could', 'through', 'had', 'o', 'when', 'such', 'himself', 'd', 'how', 'doesn', 'too', 'this', 'yourselves', 'here', 'was', 'our', 'very', 'only', 'aren', 'being', 'to', 'if', "isn't", 'no', 'again', 'they', 'is', "doesn't", 'it', 'yours', 'whom', 'than', 'couldn', 'so', "hadn't", 'its', 'didn', 'mustn', 'were', 'an', 'after', 'why', 'itself', 'are', "needn't", 'before', 'weren', 'while', 'been', "shouldn't", 'does', 'most', 'now', 'do', 'over', 'him', 'these', 'will', 'as', 'during', 'would', 'until', 'don', 'into', 'ourselves', 'other', 'ours', "wouldn't", "weren't", 'but', "don't", "aren't", 'the', 'from', 'some', 'nor', 'their', 'he', "haven't", 't', "she's", 'which', "you'll", 'between', 'who', 'your', "should've", "couldn't", "wasn't", 'in', 'and', 'more', 'theirs', 'at', 'should', 'up', 'hadn', "mightn't", 'few', 'shan', 'then', 'there', 'needn', 'my', 'because', "mustn't", 'out', 'have', 're', "you're", 'her', 'that', 'above'

In [4]:
'''-----------------------------Data Cleaning and Preprocessing pipeline----------------------------------'''

#Removing special character
def remove_special_character(content):
    return re.sub('\W+',' ', content )#re.sub('\[[^&@#!]]*\]', '', content)

# Removing URL's
def remove_url(content):
    return re.sub(r'http\S+', '', content)

#Removing the stopwords from text
def remove_stopwords(content):
    clean_data = []
    for i in content.split():
        if i.strip().lower() not in stop_words and i.strip().lower().isalpha():
            clean_data.append(i.strip().lower())
    return " ".join(clean_data)

# Expansion of english contractions
def contraction_expansion(content):
    content = re.sub(r"won\'t", "would not", content)
    content = re.sub(r"can\'t", "can not", content)
    content = re.sub(r"don\'t", "do not", content)
    content = re.sub(r"shouldn\'t", "should not", content)
    content = re.sub(r"needn\'t", "need not", content)
    content = re.sub(r"hasn\'t", "has not", content)
    content = re.sub(r"haven\'t", "have not", content)
    content = re.sub(r"weren\'t", "were not", content)
    content = re.sub(r"mightn\'t", "might not", content)
    content = re.sub(r"didn\'t", "did not", content)
    content = re.sub(r"n\'t", " not", content)
    '''content = re.sub(r"\'re", " are", content)
    content = re.sub(r"\'s", " is", content)
    content = re.sub(r"\'d", " would", content)
    content = re.sub(r"\'ll", " will", content)
    content = re.sub(r"\'t", " not", content)
    content = re.sub(r"\'ve", " have", content)
    content = re.sub(r"\'m", " am", content)'''
    return content

#Data preprocessing
def data_cleaning(content):
    content = contraction_expansion(content)
    content = remove_special_character(content)
    content = remove_url(content)
    
    content = remove_stopwords(content)    
    return content

In [5]:
%%time
pd.options.display.max_colwidth = 1000
#Data cleaning
df['Reviews_clean']=df['Reviews'].apply(data_cleaning)
df.head(5)

CPU times: total: 13.2 s
Wall time: 19.8 s


,Ratings,Reviews,Movies,Resenhas,Reviews_clean
0,1.0,"*Disclaimer: I only watched this movie as a conditional agreement. And I see films for free. I wouldn't be caught dead giving my hard earned money to these idiots.Well, to explain the depth of this 'film', I could write my shortest review, ever. Don't see this movie. It is by far the stupidest, lamest, most lazy, and unbelievably UNFUNNY movie I have ever seen. It is a total disaster. But since my hatred for this movie, and the others like it, extends far beyond one viewing, I think I'll go on for a bit.I don't know any of the people in the movie besides Carmen Electra, Vanessa Minnillo, and Kim Kardashian, but it doesn't matter. They're all horrible, though I think that was the point. The editing is flat out horrible, and possibly blatant continuity errors make this crapfast even crappier than I thought it would be. Now I know that these films are not supposed to be serious at all, but come on, it's film-making 101 that if someone gets a minor facial cut, it should be there in the...",Disaster Movie,"* IsenÃ§Ã£o de responsabilidade: eu sÃ³ assisti esse filme como um acordo condicional. E eu vejo filmes de graÃ§a. Eu nÃ£o seria pego morto dando meu dinheiro suado a esses idiotas. Bem, para explicar a profundidade desse 'filme', eu poderia escrever minha crÃ­tica mais curta de todos os tempos. NÃ£o vÃª este filme. Ã de longe o filme mais estÃºpido, lamenta, preguiÃ§oso e inacreditavelmente UNFUNNY que eu jÃ¡ vi. Ã um desastre total. Mas como o meu Ã³dio por este filme e por outros, se estende muito alÃ©m de uma exibiÃ§Ã£o, acho que vou continuar um pouco. NÃ£o conheÃ§o nenhuma das pessoas do filme alÃ©m de Carmen Electra, Vanessa Minnillo, e Kim Kardashian, mas isso nÃ£o importa. Eles sÃ£o todos horrÃ­veis, embora eu ache que esse seja o ponto. A ediÃ§Ã£o Ã© horrÃ­vel e, possivelmente, erros de continuidade flagrantes tornam essa porcaria ainda mais horrÃ­vel do que eu pensava. Agora eu sei que esses filmes nÃ£o devem ser sÃ©rios, mas vamos lÃ¡, Ã© o cinema 101 que se alguÃ©m f...",disclaimer watched movie conditional agreement see films free not caught dead giving hard earned money idiots well explain depth film write shortest review ever not see movie far stupidest lamest lazy unbelievably unfunny movie ever seen total disaster since hatred movie others like extends far beyond one viewing think go bit not know people movie besides carmen electra vanessa minnillo kim kardashian not matter horrible though think point editing flat horrible possibly blatant continuity errors make crapfast even crappier thought know films not supposed serious come film making someone gets minor facial cut next shot someone gets cut sword blood least cut though since narnia films get away give disaster movie pass jokes thoughtless mindless physical gags obviously take popular movies last year late well including best picture nominees know saddest thing stupid movies not care much money make many cameos sorry ass excuses films taking away jobs actors writers directors truly deserv...
1,1.0,"I am writing this in hopes that this gets put over the previous review of this ""film"". How anyone can find this slop entertaining is completely beyond me. First of all a spoof film entitled ""Disaster Movie"", should indeed be a spoof on disaster films. Now I have seen 1 (yes count them, 1) disaster film being spoofed, that being ""Twister"". How does Juno, Iron Man, Batman, The Hulk, Alvin and the Chipmunks, Amy Winehouse, or Hancock register as Disaster films? Selzterwater and Failburg once again have shown that they lack any sort of writing skill and humor. Having unfortunately been tortured with Date Movie and Epic Movie I know exactly what to expect from these two...no plot, no jokes just bad references and cheaply remade scenes from other films. Someone should have informed them that satire is more than just copy and paste from one film to another, though I shouldn't say that because some of these ac

## Feature Engineering

In [6]:
#Mapping rating data to Binary label 1 (+ve) if rating >=7 and 0 (-ve) if rating <=4 and 2 (neutral) if rating = 5 or 6
df['Label'] = df['Ratings'].apply(lambda x: '1' if x >= 7 else ('0' if x<=4 else '2'))
#Removing 
df=df[df.Label<'2']
data=df[['Reviews_clean','Label']]
print(data['Label'].value_counts())

0    60000
1    60000
Name: Label, dtype: int64


In [7]:
#Importing dependencies for feature engineering 
import sys
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from prettytable import PrettyTable
from nltk.tokenize import word_tokenize          
from nltk.stem import WordNetLemmatizer

## Lemmatization

In [8]:
# lemmatization of word 
class LemmaTokenizer(object):
    def __init__(self):
        self.wordnetlemma = WordNetLemmatizer()
    def __call__(self, reviews):
        return [self.wordnetlemma.lemmatize(word) for word in word_tokenize(reviews)]

## Vectoization with Count Vectorizer and TDIDF Vectorizer with Unigram

In [9]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,1), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,1), min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

### Feature Importance with Logistic Regression and Count Vectorizer with unigram

In [11]:
lgr = LogisticRegression()
lgr.fit(x_train_count, y_train)
lgr.score(x_test_count, y_test)
lgr.coef_[0]

i = 0
importantfeature = PrettyTable(["Feature", "Score"])
feature_names = countvect.get_feature_names_out()

for feature, importance in zip(feature_names, lgr.coef_[0]):
    if i <= 200:
        importantfeature.add_row([feature, importance])
        i = i + 1
print(importantfeature)

+----------------+-----------------------+
|    Feature     |         Score         |
+----------------+-----------------------+
|      able      |  0.10051132262577006  |
|   absolutely   |  0.04759118143385309  |
|      act       |  -0.1570881994286367  |
|     acting     |  -0.1670942724316938  |
|     action     |   0.2678714752644732  |
|     actor      |   -0.129091896571528  |
|    actress     |  -0.14910267921404985 |
|    actually    |  -0.0526722189696433  |
|      add       |  0.02593915148497202  |
|     adult      |  0.05990842542497193  |
|      age       |   0.1001634108608673  |
|     alien      |  -0.0711294771527887  |
|     almost     | -0.014782888531130462 |
|     along      |  0.12421208705563316  |
|    already     |  -0.20326876291116175 |
|      also      |   0.0980664156263565  |
|    although    |  0.19841679555048478  |
|     always     |  0.20158659651069388  |
|    amazing     |   0.842359246463363   |
|    american    |  0.05467683307241776  |
|    annoyi

### Feature Importance with TFIDF vectorizer and Logistic Regression with Unigram

In [12]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf, y_train)
lgr.score(x_test_tfidf, y_test)
lgr.coef_[0]

i = 0
importantfeature = PrettyTable(["Feature", "Score"])
feature_names = tfidfvect.get_feature_names_out()

for feature, importance in zip(feature_names, lgr.coef_[0]):
    if i <= 100:
        importantfeature.add_row([feature, importance])
        i = i + 1
print(importantfeature)

+----------------+-----------------------+
|    Feature     |         Score         |
+----------------+-----------------------+
|      able      |  0.38847445557197574  |
|   absolutely   |  0.47050280230037445  |
|      act       |  -1.0474548448231042  |
|     acting     |  -1.6664066139312284  |
|     action     |   2.6494495950154433  |
|     actor      |  -1.3612424166408037  |
|    actress     |  -1.0138673194568786  |
|    actually    |  -0.4776460210347111  |
|      add       |  0.13031165553597193  |
|     adult      |   0.6918439075638708  |
|      age       |   0.7657565003791862  |
|     alien      |  -0.33517941226026327 |
|     almost     |  -0.28870850602229786 |
|     along      |   0.7507480033762866  |
|    already     |  -1.1443226497761172  |
|      also      |   1.2246187286298837  |
|    although    |   1.2308562259862172  |
|     always     |   1.7096496754687944  |
|    amazing     |   4.790576125308981   |
|    american    |   0.4631889585221445  |
|    annoyi

### Vectorization with Count Vectorizer and TDIDF Vectorizer with Bigram

In [13]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(2,2), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(2,2),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

### Feature Importance with Logistic Regression and Count Vectorizer with Bigram

In [14]:
lgr = LogisticRegression()
lgr.fit(x_train_count, y_train)
lgr.score(x_test_count, y_test)
lgr.coef_[0]

i = 0
importantfeature = PrettyTable(["Feature", "Score"])
feature_names = countvect.get_feature_names_out()

for feature, importance in zip(feature_names, lgr.coef_[0]):
    if i <= 200:
        importantfeature.add_row([feature, importance])
        i = i + 1

print(importantfeature)

+-----------------------+-----------------------+
|        Feature        |         Score         |
+-----------------------+-----------------------+
|       acting bad      |  -1.3592826414983012  |
|      acting good      |   0.6547550628806681  |
|       acting not      |  -0.46782061995032975 |
|      action film      |   0.3699274125481679  |
|      action movie     |  0.29105955292127683  |
|     action packed     |   1.3386783052951952  |
|      action scene     |  0.17518973669821536  |
|    action sequence    |  0.10151696252168615  |
|     actor actress     |  -0.12557338892696454 |
|       actor not       |  -0.3918608062399068  |
|      almost every     |  -0.13260075331286292 |
|       along way       |  0.49088604257079405  |
|        also not       |  -0.13974956968461902 |
|      although not     |   0.4991023092501713  |
|     anything else     |  -0.3232091756466018  |
|        b movie        |  -0.09071784029432726 |
|       bad acting      |   -1.060594352905799  |


### Feature Importance with Logistic Regression and TFIDF Vectorizer with Bigram

In [15]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf, y_train)
lgr.score(x_test_tfidf, y_test)
lgr.coef_[0]

i = 0
importantfeature = PrettyTable(["Feature", "Score"])
feature_names = tfidfvect.get_feature_names_out()

for feature, importance in zip(feature_names, lgr.coef_[0]):
    if i <= 50:
        importantfeature.add_row([feature, importance])
        i = i + 1

print(importantfeature)

+-----------------------+----------------------+
|        Feature        |        Score         |
+-----------------------+----------------------+
|       acting bad      | -2.9871760534321976  |
|      acting good      |  1.443105414545713   |
|       acting not      |  -1.274469874526418  |
|      action film      |  1.2237150874358635  |
|      action movie     |  0.8443556589356437  |
|     action packed     |  3.023844928761595   |
|      action scene     | 0.47004358211870134  |
|    action sequence    | 0.29644626390549833  |
|     actor actress     | -0.26151819203681753 |
|       actor not       |   -0.9344034381743   |
|      almost every     | -0.29476710752147356 |
|       along way       |  1.0753056686648292  |
|        also not       | -0.29092975933087445 |
|      although not     |  1.1043969769896527  |
|     anything else     | -0.7966422564307268  |
|        b movie        | -0.4228615975141592  |
|       bad acting      |  -2.799472965808036  |
|        bad bad    

In [16]:
pd.options.display.max_colwidth = 1000
df[["Reviews","Ratings","Movies"]][(df['Ratings']>=9)&(df['Reviews_clean'].str.contains("bad review"))].head(100)

,Reviews,Ratings,Movies
120047,"While I wouldn't call this the greatest movie ever made, it's not anywhere near as bad as other reviewers have made it out to be. An average rating of 5 or 6 stars would be fair, but 1.5 is harsh and totally undeserved.Ring of Terror feels like an episode of The Twilight Zone stretched to an hour. In fact, it's so much like a TV show that one wonders if it might not have been originally created as a pilot.If you're a fan of 1950s horror/suspense series like Thriller, The Veil, One Step Beyond, Tales of Tomorrow, and Alfred Hitchcock Presents, you'll likely find this a pleasant way to spend an hour, as I did.Normally I would only give this film 6 out of 10 stars, but because others have been panning it so unmercifully, I'm giving it a 9.",9.0,Ring of Terror
120211,"This movie was a blast for my little guys, they loved every minute of it, I have read all of the bad reviews, and could not disagree more. This movie, is pure and good. There is just enough action to keep the kids interested, and not so much that you leave the theater with them bouncing off the walls either. It is funny with jokes that everyone can appreciate. I think people have gotten used to so much violence and adult content in our kids movies that they are disappointed when it is missing, like the movie wasn't entertaining enough for the parents. Well, NEWS FLASH.....It's a kids movie, and a perfect one at that. Kids need these kinds of movies, not Spongebob and the like which are more to entertain the parents.",9.0,Doogal
120238,"I am a huge horror buff and prefer pieces that delve into the characters psychological issues. This film was awesome on so many levels, the acting and writing were fantastic and creepy and I was afraid or and empathetic with the murderer the whole time. What an interesting study on the line between sick and a danger to others, and the line between being a mean girl and being psychotic. Set in a great location, a house full of creepy art, in the winter in Conneticut and with amazing performances from many of my favorite actresses. It actually shocks me that others have given this such a bad review, I loved this movie, I guess it goes to show you everyone will have a different opinion but I say don't miss this film!",9.0,#Horror
120239,"No idea why there are so many bad reviews here? I loved it; I thought it was a very advanced thoughtful film. The graphic were #killer. The comparison of video game culture and young girl culture was spot on. This film makes connections that I've never seen on the big screen but, do see in every day life.The casting was spot on, Hello 12 year-old girls are supposed to be a little annoying. I do wish that more directors would take color into more consideration the way this film does. T The highly stylized sets make the murder scenes more believable because everything is so unbelievable. How can you live in 2016 and not ""get""a film about social media and accelerationism. #duh Someone explain this to me.",9.0,#Horror
120273,"What do you get when you cross Love Story with Star Wars with Blade Runner with Back to the Future with MTV? Love Story 2050, that's what. What a fun movie for the entire family. This fantasy of epic proportions is much, much better than AI, a similar sci-fi classic. The thrills are non-stop in this blockbuster, from its lead off car chase to bike racing stunts to the vantage point of a moving roller coaster to speeding hover-craft--you will be on the edge of your seat from beginning to end. The version I saw was only partially in English and I still was glued to the screen. I can't wait to see a version with subtitles. The mega budget special effects are out of this world and highly convincing. The future vision of XBox was hilarious. Those who are complaining about how long this movie is simply don't understand Bollywood. The three hours went by quickly; it seemed to be only an hour. There could have been a better twist with the Darth Vader character. 

### Vectorization with Count Vectorizer and TFIDF Vectorizer with Trigram

In [17]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(3,3), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(3,3),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

### Feature Importance with Logistic Regression and Count Vectorizer with Trigram

In [18]:
lgr = LogisticRegression()
lgr.fit(x_train_count, y_train)
lgr.score(x_test_count, y_test)
lgr.coef_[0]

i = 0
importantfeature = PrettyTable(["Feature", "Score"])
feature_names = countvect.get_feature_names_out()

for feature, importance in zip(feature_names, lgr.coef_[0]):
    if i <= 200:
        importantfeature.add_row([feature, importance])
        i = i + 1

print(importantfeature)

+---------------------------+-----------------------+
|          Feature          |         Score         |
+---------------------------+-----------------------+
|       acting not bad      |  -0.4160544418297832  |
|      acting not good      |  -1.0608422808773543  |
|      acting not great     |  -0.4673938491427938  |
|     acting pretty good    |   0.9604427712580944  |
|      acting top notch     |   1.8064791646058318  |
|      action movie not     |   1.028407244111418   |
|      action set piece     |   1.216645783744439   |
|     action take place     |  0.042476537754819604 |
|       actor good job      |   0.5960771140401331  |
|    actually pretty good   |   0.3068308897636772  |
|    actually quite good    |  -0.16411629095462138 |
|     almost every scene    |  -0.31906905025068416 |
|        b movie not        |  -0.0472585518420228  |
|       bad acting bad      |   -2.328241263343194  |
|        bad bad bad        |  -1.7686137576847263  |
|        bad guy not        

### Feature Importance with Logistic Regression and TFIDF Vectorizer with Trigram

In [19]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf, y_train)
lgr.score(x_test_tfidf, y_test)
lgr.coef_[0]

i = 0
importantfeature = PrettyTable(["Feature", "Score"])
feature_names = tfidfvect.get_feature_names_out()

for feature, importance in zip(feature_names, lgr.coef_[0]):
    if i <= 200:
        importantfeature.add_row([feature, importance])
        i = i + 1

print(importantfeature)

+---------------------------+------------------------+
|          Feature          |         Score          |
+---------------------------+------------------------+
|       acting not bad      |  -0.5732816864239494   |
|      acting not good      |  -1.3107700909017934   |
|      acting not great     |  -0.6602718882768086   |
|     acting pretty good    |   1.0932579289986455   |
|      acting top notch     |   2.074649261245995    |
|      action movie not     |   1.204591359705366    |
|      action set piece     |   1.2430029177920827   |
|     action take place     |  0.09013657404651272   |
|       actor good job      |   0.6812433617495375   |
|    actually pretty good   |  0.33819941787645164   |
|    actually quite good    |  -0.14193988072195712  |
|     almost every scene    |  -0.3818008160086641   |
|        b movie not        |  -0.09272409100081438  |
|       bad acting bad      |   -2.858563396267401   |
|        bad bad bad        |   -2.593606458214657   |
|        b

### Vectorization with Count Vectorizer and TDIDF Vectorizer with 4-gram

In [20]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(4,4), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(4,4),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

### Feature Importance with Logistic Regression and Count Vectorizer with 4-gram

In [21]:
lgr = LogisticRegression()
lgr.fit(x_train_count, y_train)
lgr.score(x_test_count, y_test)
lgr.coef_[0]

i = 0
importantfeature = PrettyTable(["Feature", "Score"])
feature_names = countvect.get_feature_names_out()

for feature, importance in zip(feature_names, lgr.coef_[0]):
    if i <= 200:
        importantfeature.add_row([feature, importance])
        i = i + 1

print(importantfeature)

+------------------------------------------------+-----------------------+
|                    Feature                     |         Score         |
+------------------------------------------------+-----------------------+
|         according dvd sleeve synopsis          |  -2.2545695463347033  |
|           acting also great special            |  0.35749873395201415  |
|             acting bad script bad              |  -1.0274211884404705  |
|     addictive consistently compelling show     |  0.07279038856780506  |
|                 ai not gon na                  |  -0.4873083280289842  |
|           also great special effect            |   0.7705316686994392  |
|       annoying illogical decision making       |  -0.03640155113569731 |
|           another film seen recently           |  -0.03886544318346905 |
|         appreciation horror low budget         |  -0.10976598356723974 |
|        atmosphere dreary due excessive         |  -0.04064045945356016 |
|         atmospheric bea

### Feature Importance with Logistic Regression and TDIDF Vectorizer with 4-gram

In [22]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf, y_train)
lgr.score(x_test_tfidf, y_test)
lgr.coef_[0]

i = 0
importantfeature = PrettyTable(["Feature", "Score"])
feature_names = tfidfvect.get_feature_names_out()

for feature, importance in zip(feature_names, lgr.coef_[0]):
    if i <= 200:
        importantfeature.add_row([feature, importance])
        i = i + 1

print(importantfeature)

+------------------------------------------------+-----------------------+
|                    Feature                     |         Score         |
+------------------------------------------------+-----------------------+
|         according dvd sleeve synopsis          |  -2.2728234863731225  |
|           acting also great special            |   0.6744187057552073  |
|             acting bad script bad              |  -1.3135142386543972  |
|     addictive consistently compelling show     |  0.24700512847222267  |
|                 ai not gon na                  |  -0.4753392370386222  |
|           also great special effect            |   1.0330901038074507  |
|       annoying illogical decision making       |  -0.22585405492655158 |
|           another film seen recently           |  -0.24559663416104233 |
|         appreciation horror low budget         |  -0.27657853294949447 |
|        atmosphere dreary due excessive         |  -0.2585822875328966  |
|         atmospheric bea

### Vectorization with Count Vectorizer and TDIDF Vectorizer with unigram, bigram and trigram

In [23]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,3), min_df=10,max_features=5000)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,3),min_df=10,max_features=5000)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

### Feature Selection with Chi squared

In [24]:
from sklearn.feature_selection import chi2
import numpy as np
N = 5000
Number = 1
featureselection = PrettyTable(["Unigram", "Bigram","Trigram"])
for category in train['Label'].unique():
    features_chi2 = chi2(x_train_tfidf, train['Label'] == category)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidfvect.get_feature_names_out())[indices]
    unigrams = [x for x in feature_names if len(x.split(' ')) == 1]
    bigrams = [x for x in feature_names if len(x.split(' ')) == 2]
    trigrams = [x for x in feature_names if len(x.split(' ')) == 3]
    print("%s. %s :" % (Number,category))
    print("\t# Unigrams :\n\t. %s" %('\n\t. '.join(unigrams[-N:])))
    print("\t# Bigrams :\n\t. %s" %('\n\t. '.join(bigrams[-N:])))
    print("\t# Trigrams :\n\t. %s" %('\n\t. '.join(trigrams[-N:])))
    Number += 1

1. 1 :
	# Unigrams :
	. spirited
	. jail
	. returning
	. survivor
	. nicole
	. deliberately
	. huge
	. ashley
	. oil
	. member
	. pre
	. appearance
	. join
	. forth
	. claude
	. characterization
	. land
	. exorcism
	. finding
	. spite
	. inspiration
	. reporter
	. steven
	. flying
	. tie
	. item
	. speaks
	. religious
	. twin
	. fairly
	. energy
	. freak
	. jay
	. posse
	. rap
	. halloween
	. football
	. hook
	. starring
	. second
	. chief
	. rank
	. judging
	. vacation
	. fare
	. air
	. chasing
	. broken
	. remaining
	. club
	. grey
	. starting
	. colour
	. heavy
	. comparison
	. professional
	. meaningful
	. confusion
	. needed
	. ant
	. journalist
	. believed
	. shadow
	. african
	. wow
	. saturday
	. exchange
	. argue
	. person
	. traveling
	. virus
	. occur
	. numerous
	. river
	. quick
	. usa
	. tower
	. develop
	. whenever
	. road
	. decides
	. laura
	. training
	. butt
	. dick
	. birthday
	. exaggerated
	. strike
	. sea
	. arrive
	. pregnant
	. wise
	. prior
	. boot
	. accident